In [1]:
import requests
def connected_to_internet(url='http://www.google.com/', timeout=5):
    try:
        _ = requests.get(url, timeout=timeout)
        return True
    except requests.ConnectionError:
        print("No internet connection available.")
    return False

connected_to_internet()

True

In [2]:
#!pip install -r requirements.txt
#!pip install fxcmpy
#!pip install forexconnect
#!pip install sklearn
#!pip install statsmodels
#!pip install matplotlib
#!pip install ipywidgets

In [3]:
import os
import threading
import time
import importlib
import sys
import warnings
mpath = os.getcwd() + '/lib'
sys.path.append(mpath)

In [20]:
#from forexconnect import fxcorepy
import fxcmpyi
import pandas as pd
import matplotlib

import SharedObjects
import pairs
importlib.reload(SharedObjects)
importlib.reload(fxcmpyi)
importlib.reload(pairs)
#importlib.reload(sklearn)

import numpy as np

In [5]:
#fxcmpy.__version__

minutes: m1, m5, m15 and m30,
hours: H1, H2, H3, H4, H6 and H8,
one day: D1,
one week: W1,
one month: M1.

In [6]:
#Set variables
T = 'm1' #Period
sz = 300; #Size
prs = dict()
pr_mlreg = dict()
mlr = pd.DataFrame()

Get access token from https://tradingstation.fxcm.com.

demo - for account type demo

real - for account type real

In [7]:
con = None
def custom():
    global con
    try:
        def login():
            global con
            TOKEN = '2720337434ea557dea25bad4cd9ad392d70f620d'
            con = fxcmpyi.fxcmpy(access_token=TOKEN, log_level='error', server='demo')
        threading.Thread(target=login).start()
    except:
        custom()
custom()
#con.close()

In [11]:
if con is not None:
    print(con.is_connected())
    shd_obj = SharedObjects.SharedObjects(con)

True


In [21]:
#Function to update data
def price_update(data, dataframe):
    symbol = data['Symbol']
    if (len(prs) > 1):
        pr_mlreg[symbol] = prs.get(symbol).multiple_linear_regression(prs)
        mlr = shd_obj.mergeall_byrow(pr_mlreg)
        sp_ = prs.get(symbol).co_integration(prs)
        shd_obj.set_var(sp_[0], sp_[1])

In [28]:
try:
    def get_past_prices():
        for symbol in shd_obj.tradable_pairs:
            dtf = shd_obj.historical_data(symbol, T, sz)
            if symbol not in prs:
                prs[symbol] = pairs.Pair(symbol, dtf)
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    #shd_obj.stream_data(symbol, (price_update, prs.get(symbol).on_price_update) )
    threading.Thread(target=get_past_prices).start()
except:
    print("Error: unable to start thread")

In [26]:
#shd_obj.unstream_data()

In [434]:
#con.close()

In [ ]:
#status = shd_obj.get_status()
#if(status == 'You are logged in'):
    #shd_obj.stream_data(T, sz, price_update)

In [ ]:
##Reverses the dataframe rows
#df[::-1]

#c = df.drop(columns=['tickvolume'])
#dates[0].minute
#dates[0].hour

#con.is_subscribed(instruments[0])
#con.get_subscribed_symbols()

#con.unsubscribe_market_data('EUR/USD')
#print(shd_obj.pairs)
#con.close()

In [ ]:
#con.get_default_account()
#con.get_prices('EUR/USD')

In [ ]:
#import inspect
#source_file_DF = inspect.getsource(pd.concat)
#print(source_file_DF)